## Zomato Bangalore Restaurants
Bengaluru(Bangalore) is the capital and largest city of the Indian state of Karnataka. Bangalore is referred to as the “Silicon Valley of India” because of its role as the nation’s leading information technology(IT) exporter.

Bangalore has a unique food culture. Restaurants from all over the world can be found here in Bengaluru, with various kind of cuisines.

The food industry is always at a rise in Bangalore, with 12,000 plus restaurants currently active in the city, the number is still increasing. The growing number of restaurants and dishes in Bangalore is what attracts me to inspect the data to get some insights, some interesting facts and figures.

![](https://vignette.wikia.nocookie.net/logopedia/images/9/9b/Zomato.png/revision/latest/scale-to-width-down/340?cb=20200321171709)


Zomato is an Indian restaurant aggregator and food delivery start-up founded by Deepinder Goyal and Pankaj Chaddah in 2008. Zomato provides information, menus and user-reviews of restaurants as well as food delivery options from partner restaurants in select cities.

* Industry :	Online food ordering Retail
* Services :	Restaurant Search & Discovery, Online Ordering, Table Reservations & Management, POS Systems, Subscription Service
* Employees:	5,000+

#  Lets dive into the Bangalore's Zomato Restaurants

Data Description

- url : This feature contains the url of the restaurant on the Zomato website
- address : This feature contains the address of the restaurant in Bangalore
- name : This feature contains the name of the restaurant
- online_order : whether online ordering is available in the restaurant or not
- book_table : table book option available or not
- rate : contains the overall rating of the restaurant out of 5
- votes : contains total number of upvotes for the restaurant
- phone : contains the phone number of the restaurant
- location : contains the neighborhood in which the restaurant is located
- rest_type : restaurant type
- dish_liked : dishes people liked in the restaurant
- cuisines : food styles, separated by comma
- approx_cost(for two people) : contains the approximate cost of meal for two people
- reviews_list : list of tuples containing reviews for the restaurant, each tuple consists of two values, rating and review by the customer
- menu_item : contains list of menus available in the restaurant
- listed_in(type) : type of meal
- listed_in(city) : contains the neighborhood in which the restaurant is located

### Working on Dataset

In [ ]:
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import xgboost
from plotly.offline import plot, iplot, init_notebook_mode
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
from statsmodels.formula.api import ols
init_notebook_mode(connected=True)
import plotly
plotly.offline.init_notebook_mode (connected = True)

warnings.filterwarnings("ignore")

%matplotlib inline
ds=pd.read_csv("../input/zomato-bangalore-restaurants/zomato.csv")

In [ ]:
#Renaming the columns
ds=ds.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type','listed_in(city)':'city'})

ds.columns

### Restaurants delivering Online or not

In [ ]:
sns.countplot(ds['online_order'])
fig=plt.gcf()
fig.set_size_inches(10,10)
plt.title('Restaurants delivering online or not')

#### Pie chart

In [ ]:
labels = ["Accepted",'Not Accepted']
values = ds['online_order'].value_counts().values
fig1, ax1 = plt.subplots()
colors = ['orange', 'skyblue']
ax1.pie(values, labels=labels, autopct='%1.1f%%',shadow=True,startangle=90,colors=colors)
plt.title('Online order')
plt.show()

### Location wise count for restaurants

In [ ]:
sns.countplot(ds['city'])
sns.countplot(ds['city']).set_xticklabels(sns.countplot(ds['city']).get_xticklabels(), rotation=90, ha="right")
fig = plt.gcf()
fig.set_size_inches(13,13)
plt.title('Location wise count for restaurants')

#### No. of Restaurants in a Location

In [ ]:
fig = plt.figure(figsize=(20,7))
loc = sns.countplot(x="location",data=ds, palette = "Set1")
loc.set_xticklabels(loc.get_xticklabels(), rotation=90, ha="right")
plt.ylabel("Frequency",size=15)
plt.xlabel("Location",size=18)
loc
plt.title('No. of restaurants in a Location',size = 20,pad=20)

#### Top 5 Restaurants in Bangalore based on the Highest voting and Ratings

In [ ]:
ds["rate"]=ds["rate"].fillna(0)
li=[]
for i in range(0,len(ds)):
    a=ds.iloc[i,5]
    try:
        b=float(a[:-2])
        li.append(b)
    except:
        li.append(0)
ds["rate"]=li
res=ds.nlargest(30,["votes","rate"])
grp=res.groupby("name")
res=grp.mean()

fig = px.bar(res, y=res.rate, x=res.index, text=res.votes)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()

📌
*Restaurants Names*
* AB's -Absolute Barbecues - 4.8 rate(mean) 
* Byg Brewski Brewing Company - 4.9 rate(mean) 
* The Black Pearl - 4.7 rate(mean) 
* Toit - 4.7 rate(mean) 
* Truffles - 4.7 rate(mean)

# 📌 Sentiment Analysis



Sentiment analysis is the interpretation and classification of emotions (positive, negative and neutral) within text data using text analysis techniques. Sentiment analysis tools allow businesses to identify customer sentiment toward products, brands or services in online feedback

In [ ]:
from textblob import TextBlob

In [ ]:
sent=[]
for i in range(0,len(ds)):
    analysis = TextBlob(ds.loc[i,"reviews_list"])
    if analysis.sentiment.polarity > 0:
        sent.append("positive")
    elif analysis.sentiment.polarity == 0:
        sent.append("Neutral")
    else:
        sent.append("negative")
ds["reviews_list"]=sent

In [ ]:
ds["Count"]=1
pos=ds[ds["reviews_list"]=="positive"]
neg=ds[ds["reviews_list"]=="negative"]
cnt_pos=pos.groupby("type").sum()
cnt_neg=neg.groupby("type").sum()

## 📌 Overall Review Report

In [ ]:
fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number+gauge+delta", value = 0,
    delta = {'reference': 0},
    domain = {'x': [0.25, 1], 'y': [0.08, 0.25]},
    title = {'text': "Neutral"},
    gauge = {
        'shape': "bullet",
        'axis': {'range': [None, 300]},
        'threshold': {
            'line': {'color': "black", 'width': 2},
            'thickness': 0.75,
            'value': 170},
        'steps': [
            {'range': [0, 150], 'color': "gray"},
            {'range': [150, 250], 'color': "lightgray"}],
        'bar': {'color': "black"}}))

fig.add_trace(go.Indicator(
    mode = "number+gauge+delta", value = 4324,
    delta = {'reference': 200},
    domain = {'x': [0.25, 1], 'y': [0.4, 0.6]},
    title = {'text': "Negative"},
    gauge = {
        'shape': "bullet",
        'axis': {'range': [None, 5000]},
        'threshold': {
            'line': {'color': "black", 'width': 2},
            'thickness': 0.75,
            'value': 50},
        'steps': [
            {'range': [0, 200], 'color': "gray"},
            {'range': [200, 500], 'color': "lightgray"}],
        'bar': {'color': "black"}}))

fig.add_trace(go.Indicator(
    mode = "number+gauge+delta", value = 39629,
    delta = {'reference': 200},
    domain = {'x': [0.25, 1], 'y': [0.7, 0.9]},
    title = {'text' :"Positive"},
    gauge = {
        'shape': "bullet",
        'axis': {'range': [None, 51717]},
        'threshold': {
            'line': {'color': "black", 'width': 2},
            'thickness': 0.75,
            'value': 210},
        'steps': [
            {'range': [0, 25000], 'color': "gray"},
            {'range': [25000, 51717], 'color': "lightgray"}],
        'bar': {'color': "black"}}))
fig.update_layout(height = 400 , margin = {'t':0, 'b':0, 'l':0})

fig.show()

## 📌 Sentiment Analysis on Restaurant Type 📊

In [ ]:
labels = list(cnt_pos.index)

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=labels, values=list(cnt_pos.Count), name="Positive"),
              1, 1)
fig.add_trace(go.Pie(labels=labels, values=list(cnt_neg.Count), name="Negative"),
              1, 2)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+percent+name")

fig.update_layout(
    annotations=[dict(text='Positive', x=0.18, y=0.5, font_size=20, showarrow=False),
                 dict(text='Negative', x=0.82, y=0.5, font_size=20, showarrow=False)])
fig.show()

📌
* On the Positive side , for Delivery and Dine-out has got much positive review

##### Whilst

* On the Negative side , 62% is been given negative feedback for the Delivery

* So , there is really something problem with the Delivery side

## 📌 Types of Zomato Restaurants in Bangalore

In [ ]:
to=ds.groupby(["type","name"]).sum()

lis=list(to.index)

val1=[]
for i in range(0,len(lis)):
    val1.append(lis[i][0])


st=set(val1)

type_rest=[]
cnt=[]
for i in st:
    type_rest.append(i)
    cnt.append(val1.count(i))
    
    

type_rest,cnt
dff=pd.DataFrame()
dff["Types of Zomato Restaurants in Bangalore"]=type_rest
dff["Count of Restaurants in Bangalore"]=cnt
fig = px.bar(dff, x="Types of Zomato Restaurants in Bangalore", y="Count of Restaurants in Bangalore",
         height=400)
fig.show()

📌
*            Desserts -1009
*               Buffet - 281
*                 Cafes - 555
*              Delivery -6636
*    Drinks & nightlife - 382
*         Pubs and bars -317
*              Dine-out -6379

## 📌 Top 5 Restaurants with low food cost  based on Type 🌮 🌯 🥗

In [ ]:
cost=[]
for i in range (0,len(ds)):
    it=ds.loc[i,"cost"]
    try:
        a=it.replace(",","")
        a=float(a)
        cost.append(a)
    except:
        cost.append(it)
ds["cost"]=cost
top_cost=ds.nsmallest(15,"cost")
top_cst=top_cost.loc[:,["dish_liked","cost","type","city","name"]]

In [ ]:
fig = px.funnel(top_cst, x='cost', y='name', color='type')
fig.show()

## 📌 Top 5 Restaurants with High food cost based on Type 🌭 🍔 🍟 💵

In [ ]:
cost=[]
for i in range (0,len(ds)):
    it=ds.loc[i,"cost"]
    try:
        a=it.replace(",","")
        a=float(a)
        cost.append(a)
    except:
        cost.append(it)
ds["cost"]=cost
top_cost=ds.nlargest(10,"cost")
top_cst=top_cost.loc[:,["dish_liked","cost","type","city","name"]]

In [ ]:

fig = px.funnel(top_cst, x='cost', y='name', color='type')
fig.show()

## 📌 Ratings based on the types of Restaurants

In [ ]:
fin=ds.loc[:,["rate","type","cost"]]

In [ ]:
fig = px.histogram(fin, x="rate", color="type")
fig.show()

📌
* Rating spread is looks some how equally with all types , there are not specific discrimination on the rating based on the type

## 📌 Rating based on the Review list

In [ ]:
df = px.data.gapminder()
df_2007 = df.query("year==2007")
fig = px.scatter(ds,
                     x="rate", y="cost", size="votes", color="reviews_list",
                     log_x=True, size_max=60,
                     template="plotly_dark")
fig.show()

📌
* This plot cleary says , 
        * Negative review cases are gotten less rating , which ensures the reliability of the data.

## End of the Notebook